# Amplicon Sequence Data Analysis with QIIME 2

Adding ```!``` before the command tells the notebook this is a bash command, rather than python.

To use sequencing data in QIIME2, we first need to turn the FASTQ files containing our data into QIIME artifacts.

What the QIIME2 pipeline will do:
![our workflow](https://github.com/Gibbons-Lab/isb_course_2023/raw/main/docs/16S/assets/steps.png)

### About the Data
I downloaded FASTQ data files generated by [Mr. DNA Lab](https://www.mrdnalab.com/) Molecular Research. 
I unzipped the folders and uploaded the Mr. DNA analysis pipeline files into the `coral-pae-temp/analysis/microbiome/mrdna` directory, and the `sample-metadata.tsv` and `demux` folder into `coral-pae-temp/analysis/microbiome/rawdata` directory.  

Here I am working with the FASTQ data files located in `coral-pae-temp/analysis/microbiome/rawdata/demux`. In the `demux` folder are two `fastq.gz` files for each of the 22 samples, one for the forward read and one for the reverse read. 

The `fastq.gz` file name includes the sample identifier and should look like `4.Ea_S1_L001_R1_001.fastq.gz`. 
The underscore-separated fields in this file name are:

1.  the sample identifier,

2.  the barcode sequence or a barcode identifier,

3.  the lane number,

4.  the direction of the read (i.e. R1 or R2, because these are paired-end reads), and

5.  the set number.
   

The `fastq.gz` files are **Demultiplexed** (aka **Demuxed**) sequences that still have the forward and reverse primers in the sequences.

-   The Raw Data is **demultiplexed**

-   A R1 and R2 fastq.gz file has been generated for each individual sample

-   All forward reads are binned into the R1 fastq.gz files

-   All reverse reads are binned into the R2 fastq.gz files

-   Other than demultiplexing; you can consider the Raw Data on BaseSpace as untouched (**The Forward and Reverse Primer Sequences have not been removed**)

## Python 3 API import qiime plugins

In [3]:
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import 

In [46]:
#pip install empress
#!qiime dev refresh-cache
#!qiime empress --help

Usage: qiime empress [OPTIONS] COMMAND [ARGS]...

  Description: This QIIME 2 plugin wraps Empress and supports interactive
  visualization of phylogenetic trees.

  Plugin website: http://github.com/biocore/empress

  Getting user support: Please post to the QIIME 2 forum for help with this
  plugin: https://forum.qiime2.org

Options:
  --version            Show the version and exit.
  --example-data PATH  Write example data and exit.
  --citations          Show citations and exit.
  --help               Show this message and exit.

Commands:
  community-plot  Visualize phylogenies and community data with Empress (and,
                  optionally, Emperor)
  tree-plot       Visualize phylogenies with Empress


## Take a Look at the Metadata
Make a table of the metadata.
Here I added columns 'Pae', 'Temp', 'PeaTemp', 'Colony', and 'Tank' to the original `sample-metadata.tsv` file provided to me by Mr. DNA and renamed it `sample-metadata-verbose.tsv`
This was a bit of a process... I had to:
upload the `sample-metadata.tsv` to Excel
edit the metadata by adding the above columns and values
save it as a csv
open it in a text editor
search for all ',' commas, and find&replace them with 'TAB' symbols
save as a tab separated file `.tsv`
upload it back into the `coral-pae-temp/analysis/microbiome/rawdata` folder

:::{.callout-important}
At first I had named the new columns 'pae', 'temp', etc. with lower case... for some reason this was a problem later on and the interactive emperor plots wouldn't recognize the new columns. When I changed the column names to CamelCase to match the others, it worked. The `qiime2` docs indicate that metadata formatted with an Identifier Column such as `#Sample ID` is [case-sensitive](https://docs.qiime2.org/2023.5/tutorials/metadata/#metadata-formatting-requirements:~:text=feature%2Did-,Case%2Dsensitive,-(these%20are%20mostly)
:::

In [1]:
!qiime metadata tabulate \
  --m-input-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/metadata-verbose.qzv

Saved Visualization to: ../output/metadata-verbose.qzv


In [4]:
Visualization.load('../output/metadata-verbose.qzv')

<visualization: Visualization uuid: 357ae5fb-826d-403e-969d-ec023b3d7f34>

# Import Sequences into QIIME2

Here I follow the QIIME2 [Casava 1.8 paired-end demultiplexed fastq](https://docs.qiime2.org/2023.5/tutorials/importing/#:~:text=Casava%201.8%20paired%2Dend%20demultiplexed%20fastq) tutorial example on importing data

!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path ../rawdata/demux \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path ../output/demux-paired-end.qza

The `demux-paired-end.qza` artifact contains raw, demultiplexed sequences that still have forward and reverse primers

## Trim primers from paired-end sequences using `cutadapt`

> "The PCR primers (F515/R806) were developed against the V4 region of the 16S rRNA, which we determined would yield optimal community clustering with reads of this length using a procedure similar to that of ref. 15. [For reference, this primer pair amplifies the region 533–786 in the Escherichia coli strain 83972 sequence (greengenes accession no. prokMSA_id:470367).] The reverse PCR primer is barcoded with a 12-base error correcting Golay code to facilitate multiplexing of up to ≈1,500 samples per lane, and both PCR primers contain sequencer adapter regions." - (Caporasco et al. 2011)

Caporaso, J. G., Lauber, C. L., Walters, W. A., Berg-Lyons, D., Lozupone, C. A., Turnbaugh, P. J., Fierer, N., & Knight, R. (2011). Global patterns of 16S rRNA diversity at a depth of millions of sequences per sample. Proceedings of the National Academy of Sciences, 108(supplement_1), 4516–4522. https://doi.org/10.1073/pnas.1000080107

> "The V4 variable region of the 16S rRNA gene was amplified using the 515F (5′-­GTGCCAGCMGCCGCGGTAA-­3′) and 806R (5′-­GGACTACHVGGGTWTCTAAT-­3′) primer set (Caporaso et al. 2011). ” - (Brown et al. 2021)

Brown, Tanya, Dylan Sonett, Jesse R. Zaneveld, and Jacqueline L. Padilla-Gamiño. 2021. “Characterization of the Microbiome and Immune Response in Corals with Chronic Montipora White Syndrome.” Molecular Ecology 30 (11): 2591–2606. https://doi.org/10.1111/mec.15899.

In [5]:
!qiime cutadapt trim-paired \
  --i-demultiplexed-sequences ../output/demux-paired-end.qza \
  --p-cores 4 \
  --p-front-f GTGYCAGCMGCCGCGGTAA \
  --p-front-r GGACTACNVGGGTWTCTAAT \
  --o-trimmed-sequences ../output/demux-trimmed.qza

Saved SampleData[PairedEndSequencesWithQuality] to: ../output/demux-trimmed.qza


## Visualize trimmed & demultiplexed sequences

In [6]:
!qiime demux summarize \
  --i-data ../output/demux-trimmed.qza \
  --o-visualization ../output/demux-trimmed-summary.qzv

Saved Visualization to: ../output/demux-trimmed-summary.qzv


We are looking for at least 10,000 reads per sample. We have a minimunm of 205, 508 reads, so we are well above the minimal threshold for 'enough' reads to work with. Some of these reads will be removed in the downstream processing when we 'denoise' using the DADA2 algorithm to remove low quality reads, amplicon sequence variants, and chimeras.

In [7]:
Visualization.load('../output/demux-trimmed-summary.qzv')

<visualization: Visualization uuid: 2239e507-40ef-48de-b79d-77d9ae53efe3>

## [Denoise with DADA2](https://docs.qiime2.org/2023.5/tutorials/moving-pictures/#sequence-quality-control-and-feature-table-construction:~:text=with%20QIIME%201.-,Option%201%3A%20DADA2%C2%B6,-DADA2%20is%20a)

[DADA2](https://pubmed.ncbi.nlm.nih.gov/27214047/) is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. 

As implemented in the q2-dada2 plugin, this quality control process will additionally filter any phiX reads (commonly present in marker gene Illumina sequence data) that are identified in the sequencing data, and will filter chimeric sequences.  
The dada2 denoise-paired method requires four parameters that are used in quality filtering:  

    `--p-trim-left-f m`, which trims off the first m bases of each sequence in the forward reads
    `--p-trim-left-r n`, which trims off the first m bases of each sequence in the reverse reads
    `--p-trunc-len-f o`, which truncates each sequence at position o in the forward reads
    `--p-trunc-len-f p`, which truncates each sequence at position p in the reverse reads  
    
This allows the user to remove low quality regions of the sequences.  

What is a 'good' quality score?  

In QIIME 2's interactive quality plots, the quality scores typically range from 0 to 40. Quality scores reflect the accuracy of base calls in sequencing data, with higher scores indicating higher accuracy. The most common quality score scale used in modern sequencing technologies is the Phred scale.

In the Phred sceal - 

A quality score of 10 corresponds to a 1 in 10 chance of an incorrect base call (90% aacc  y).
A quality score of 20 corresponds to a 1 in 100 chance of an incorrect base call (99%r ua  cy).
A quality score of 30 corresponds to a 1 in 1000 chance of an incorrect base call  (9curac9.9%
accur
A "good" quality score in this context depends on your specific analysis goals and the sequencing platform you're u
sing. However, many researchers consider quality s3b and aboveove 20 to be generally acceptable for downstreaanal**ysis. Scores above 30 are often seen as very ha qu*. In theew the Interactive Quality Plot tab in the `demux-trimmed-summary.qzv` file that was generated by `qiime demux summarlots, we see that the quality scores of the bases are high, between a score of 11 in the lowest 2nd percentile and a score of 37 in the bottom 25th percentile and hitter.  

[Denoising tips from Greg Caporasco](https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/020-tutorial-upstream/040-denoising.html) using `qiime dada2 denoise-paired`

 - F orward- 230
 - R eve31

This step will take 10-15 minutes!t step

In [6]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs ../output/demux-trimmed.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 230 \
  --p-trunc-len-r 231 \
  --p-n-threads 20 \
  --output-dir ../output/dada --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/tmpwef8c0hn/forward --input_directory_reverse /tmp/tmpwef8c0hn/reverse --output_path /tmp/tmpwef8c0hn/output.tsv.biom --output_track /tmp/tmpwef8c0hn/track.tsv --filtered_directory /tmp/tmpwef8c0hn/filt_f --filtered_directory_reverse /tmp/tmpwef8c0hn/filt_r --truncation_length 230 --truncation_length_reverse 231 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 2.0 --max_expected_errors_reverse 2.0 --truncation_quality_score 2 --min_overlap 12 --pooling_method independent --chimera_method consensus --min_parental_fold 1.0 --allow_one_off False --num_threads 20 --learn_min_reads 1000000

R version 4.2.3 (2023-03-15) 
Loading required package: Rcpp
DADA2: 1.26.0 / Rcpp: 1.0.10 / RcppParallel: 5.1.6 
2) Filte

### Visualize Denoising Stats

Our denoising stats are contained in an artifact. To convert it to a visualization we can use `qiime metadata tabulate`.

In [8]:
!qiime metadata tabulate \
  --m-input-file ../output/denoising-stats.qza \
  --o-visualization ../output/dada/denoising-stats.qzv

Saved Visualization to: ../output/dada/denoising-stats.qzv


One good way to tell if the identified ASVs are representative of the sample is to see how many reads were maintained throughout the pipeline. Here, the most common issues and solutions are:

**Large fraction of reads is lost during merging (only paired-end)**

![read overlap](https://gibbons-lab.github.io/isb_course_2023/16S/assets/read_overlap.png)

In order to merge ASVs DADA2 uses an overlap of 12 bases between forward and reverse reads by default. Thus, your reads must allow for sufficient overlap *after* trimming. So if your amplified region is 450bp long and you have 2x250bp reads and you trim the last 30 bases of each read, truncating the length to 220bp, the total length of covered sequence is 2x220 = 440 which is shorter than 450bp so there will be no overlap. To solve this issue trim less of the reads or adjust the `--p-min-overlap` parameters to something lower (but not too low).

<br>

**Most of the reads are lost as chimeric**

![read overlap](https://gibbons-lab.github.io/isb_course_2023/16S/assets/chimera.png)

This is usually an experimental issue as chimeras are introduced during amplification. If you can adjust your PCR, try to run fewer cycles. Chimeras can also be introduced by incorrect merging. If your minimum overlap is too small ASVs may be merged randomly. Possible fixes are to increase the `--p-min-overlap` parameter or run the analysis on the forward reads only (in our empirical observations, chimeras are more likely to be introduced in the joined reads). *However, losing between 5-25% of your reads to chimeras is normal and does not require anytadata tabulate`.

In [9]:
Visualization.load('../output/dada/denoising-stats.qzv')

<visualization: Visualization uuid: ff8ce97e-8b05-49f7-8bfd-bc3db40d6fbd>

## Filter feature table

In [22]:
import pandas as pd

In [28]:
# read in tsv sample metadata as a csv
df = pd.read_csv('../data/sample-metadata-verbose.tsv', delimiter='\t')
# make a list of SampleID values to remove
remove = ['1.Ea', '2.Ea', '3.Eb', '4.Ea', 'blank.', 'mock.']
# remove those rows 
df = df[~df['#SampleID'].isin(remove)]
df

,#SampleID,BarcodeSequence,LinkerPrimerSequence,BarcodeName,ReversePrimer,ProjectName,Description,Pae,Temp,PaeTemp,Colony,Tank
0,1.CA2a,ATCATAGGCT,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0073,GGACTACNVGGGTWTCTAAT,060823STillcus515F,1.CA2a,control,ambient,control-ambient,1,A2
1,1.CH2a,TGTTAGAAGG,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0074,GGACTACNVGGGTWTCTAAT,060823STillcus515F,1.CH2a,control,hot,control-hot,1,H2
3,1.PA2a,ACGGCCGTCA,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0076,GGACTACNVGGGTWTCTAAT,060823STillcus515F,1.PA2a,peak,ambient,peak-ambient,1,A2
4,1.PH1a,CGTTGCTTAC,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0077,GGACTACNVGGGTWTCTAAT,060823STillcus515F,1.PH1a,peak,hot,peak-hot,1,H1
5,2.CA2a,TGACTACATA,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0078,GGACTACNVGGGTWTCTAAT,060823STillcus515F,2.CA2a,control,ambient,control-ambient,2,A2
6,2.CH1b,CGGCCTCGTT,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0079,GGACTACNVGGGTWTCTAAT,060823STillcus515F,2.CH1b,control,hot,control-hot,2,H1
8,2.PA1b,TCGTCTGACT,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0081,GGACTACNVGGGTWTCTAAT,060823STillcus515F,2.PA1b,peak,ambient,peak-ambient,2,A1
9,2.PH2a,CTCATAGCGA,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0082,GGACTACNVGGGTWTCTAAT,060823STillcus515F,2.PH2a,peak,hot,peak-hot,2,H2
10,3.CA1b,AGACACATTA,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0083,GGACTACNVGGGTWTCTAAT,060823STillcus515F,3.CA1b,control,ambient,control-ambient,3,A1
11,3.CH2a,GCGCGATGTT,GTGYCAGCMGCCGCGGTAA,60bp_UDPi5_0084,GGACTACNVGGGTWTCTAAT,060823STillcus515F,3.CH2a,control,hot,control-hot,3,H2


In [30]:
# save the new df as a tsv
df.to_csv('../data/treatment-only-metadata.tsv', sep='\t', index=False)

In [32]:
!qiime feature-table filter-samples \
  --i-table ../output/dada/table.qza \
  --m-metadata-file ../data/treatment-only-metadata.tsv \
  --o-filtered-table ../output/dada/treatment-only-table.qza

Saved FeatureTable[Frequency] to: ../output/dada/treatment-only-table.qza


## Summarize & tabulate the feature table
After the quality filtering step completes, you’ll want to explore the resulting data. You can do this using the following two commands, which will create visual summaries of the data. The `feature-table summarize` command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. The `feature-table tabulate-seqs` command will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database.

### feature-table summarize
The feature-table summarize command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. Feature tables in QIIME 2 represent the abundance of different biological features (such as bacterial taxa or OTUs) across samples.
 In this command:

--i-table tab.qzal`es the input feature table in QIIME 2 artifact format (.qza file) that you want to sumri

--o-visualizattable.qzv tfies the output visualization in QIIME 2 artifact format (.qzv file) that will con the summaes.

--m-sample-metadata-file sampldata.tsve-cifies the metadata file (usually in tab-separated values format) that contains additional information about the samples in your featble  and m

In [33]:
!qiime feature-table summarize \
  --i-table ../output/dada/treatment-only-table.qza \
  --o-visualization ../output/dada/treatment-only-table.qzv \
  --m-sample-metadata-file ../data/treatment-only-metadata.tsv

Saved Visualization to: ../output/dada/treatment-only-table.qzv


In [34]:
Visualization.load('../output/dada/treatment-only-table.qzv')

<visualization: Visualization uuid: 3fb6bd02-ea73-4612-b40b-6e7550f36c21>

### feature-table tabulate-seqs

In [12]:
!qiime feature-table tabulate-seqs \
  --i-data ../output/dada/representative_sequences.qza \
  --o-visualization ../output/dada/representative_sequences.qzv

Saved Visualization to: ../output/dada/representative_sequences.qzv


In [14]:
Visualization.load('../output/dada/representative_sequences.qzv')

<visualization: Visualization uuid: 302a519e-a5e5-41e1-8c04-41c9d10765b4>

## Diversity & Phylogenetics

[Generate a tree for phylogenetic diversity analyses](https://docs.qiime2.org/2023.5/tutorials/moving-pictures-usage/#:~:text=Generate%20a%20tree%20for%20phylogenetic%20diversity%20analyses)

From the moving pictures tutorial:
> QIIME supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the FeatureTable[Frequency] QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another. This information will be stored in a Phylogeny[Rooted] QIIME 2 artifact. To generate a phylogenetic tree we will use align-to-tree-mafft-fasttree pipeline from the q2-phylogeny plugin. 
First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [15]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences ../output/dada/representative_sequences.qza \
  --output-dir ../output/tree

Saved FeatureData[AlignedSequence] to: ../output/tree/alignment.qza
Saved FeatureData[AlignedSequence] to: ../output/tree/masked_alignment.qza
Saved Phylogeny[Unrooted] to: ../output/tree/tree.qza
Saved Phylogeny[Rooted] to: ../output/tree/rooted_tree.qza


In [48]:
!qiime empress tree-plot \
    --i-tree ../output/tree/rooted_tree.qza \
    --o-visualization ../output/tree/empress.qzv

Saved Visualization to: ../output/tree/empress.qzv


In [49]:
Visualization.load("../output/tree/empress.qzv")

<visualization: Visualization uuid: de513a54-510f-430c-94c2-ebfe151b2cbe>

[Diversity](https://docs.qiime2.org/2023.5/tutorials/moving-pictures-usage/#:~:text=Alpha%20and%20beta%20diversity%20analysis)

QIIME 2’s diversity analyses are available through the `q2-diversity` plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We’ll first apply the `core-metrics-phylogenetic` method, which rarefies a `FeatureTable Frequency` to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics.

An important metric to consider when studying microbial ecology is __diversity__. Diversity comes in two flavors: ⍺ (alpha) and β (beta).

Alpha diversity is pretty simple - how diverse is a single sample? You might consider measures like richness and evenness.

![alpha diversity](https://gibbons-lab.github.io/isb_course_2023/16S/assets/alpha_diversity.png)

Beta diversity instead looks at how different two samples are from each other - what taxa are shared, and how their abundances differ.

![beta diversity](https://gibbons-lab.github.io/isb_course_2023/16S/assets/beta_diversity.png)


The metrics computed by default are:
##### Alpha diversity- 
Shannon’s diversity index (a quantitative measure of community richness- 

Observed Features (a qualitative measure of community richne- s)

Faith’s Phylogenetic Dive**rsity (a qualitative measure of community richness that incorporates phylogenetic relationships between the fea**t- res)

Evenness (or Pielou’s Evenness; a measure of community eve##### nness)

Beta - iversity

Jaccard distance (a qualitative measure of community dis- imilarity)

Bray-Curtis distance (a quantitative measure of community d- ssimilarity)

unweighted U**niFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships betwe**e-  the features)

weighted** UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships bet**ween the features) 

### Sampling Depth

An important parameter that needs to be provided to this code is ` --p-sampling-depth`, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For am--ple, if you provide `--p-sampling-depth 500`, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the information presented in the `table.qzv` file that was created above. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.

In [35]:
## open interactive table visualization
Visualization.load('../output/dada/treatment-only-table.qzv')

<visualization: Visualization uuid: 3fb6bd02-ea73-4612-b40b-6e7550f36c21>

Navigate to the interactive sample detail tab
<br>
Move the sampling depth slider as high as you can before excluding any samples 
<br>
We want the sampling depth to be high, while retaining all 22 samples
<br>
This looks like a sampling depth of 10,6727 (09AUG2023, SST) 
<br>
But maybe we exclude the mock community and bring it up to 163971? (15AUG2023, SST)
<br>
... Redoing this with only the 16 samples that were exposed to the treatment ( so we're excluding here the mock and blank and environmental baseline samples)
<br>
In this.. the lowest frequency is 166920 

What value would you choose to pass for --p-sampling-depth? 
- **166900**
How many samples will be excluded from your analysis based on this choice? 
- **none, all 16 treatment samples are retained**
How many total sequences will you be analyzing in the core-metrics-phylogenetic command?
2,670,400 (68.21%) features present across the 16 treatment samples

<br>
The mock community has the fewest features at **107,656** and is our'limiting factor' to increase sample depth.
Why does the blank have so many features! That is not good... 

To account for variations in sampling depth, we'll provide QIIME2 with a cutoff at which rarefy all our samples. Since this randomly selects sequences, your results might look a little different. We'll also pass in our metadata file, so we can keep track how which samples come from each group.

In [37]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ../output/tree/rooted_tree.qza \
  --i-table ../output/dada/treatment-only-table.qza \
  --p-sampling-depth 166900 \
  --m-metadata-file ../data/treatment-only-metadata.tsv \
  --output-dir ../output/diversity-core

Saved FeatureTable[Frequency] to: ../output/diversity-core/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: ../output/diversity-core/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/diversity-core/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/diversity-core/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/diversity-core/evenness_vector.qza
Saved DistanceMatrix to: ../output/diversity-core/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: ../output/diversity-core/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: ../output/diversity-core/jaccard_distance_matrix.qza
Saved DistanceMatrix to: ../output/diversity-core/bray_curtis_distance_matrix.qza
Saved PCoAResults to: ../output/diversity-core/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: ../output/diversity-core/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: ../output/diversity-core/jaccard_pcoa_results.qza
Saved PCoAResu

### Unweighted Unifrac Emperor Plot

In [38]:
Visualization.load('../output/diversity-core/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 23a4bc4b-e26f-45d4-b49d-60231e9d6a88>

### Weighted Unifrac Emperor Plot

In [39]:
Visualization.load('../output/diversity-core/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 545dd2ee-a159-4601-ab42-7558cd6f804b>

### Jaccard Emperor PLot

In [40]:
Visualization.load('../output/diversity-core/jaccard_emperor.qzv')

<visualization: Visualization uuid: cea316fc-8fa6-461a-8c9f-be039d010013>

### Bray-Curtis Emperor Plot

In [41]:
Visualization.load('../output/diversity-core/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: de674f56-2a32-4b7f-af53-d6a1c83e5030>

## Alpha Diversity
After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file `../rawdata/sample-metadata-verbose.tsv`.

We’ll first test for associations between categorical metadata columns and alpha diversity data.
We’ll do that here for the Faith Phylogenetic Diversity (a measure of community richness):

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity ../output/diversity-core/faith_pd_vector.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/faith-pd-group-significance.qzv

### Microbial Community Richness: Faith Phylogenetic Diversity

##### 
Which categorical sample metadata columns are most strongly associated with the differences in microbial community richness
Temperature Treatment

##### Are these differences statistically significat?n

Pae
- no significantly different groups

Temp
- ambient (n=12) vs. hot (n=8), pvalue (0.02)
  
PaeTemp
- peak-ambient (n=4) vs. peak-hot (n=4), pvalue (0.04)
- env-ambient (n=4) vs. peak-hot (n=4), pvalue (0.04)

Colony
- no significantly different groups

Tank
- H1 (n=5) vs KB (n=4), pvalue (0.05)
- A2 (n=4) vs H1 (n=5), pvalue (0.05)t?

In [ ]:
Visualization.load('../output/faith-pd-group-significance.qzv')

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity ../output/diversity-core/evenness_vector.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/evenness-group-significance.qzv

### Microbial Community Evenness, Alpha diversity , Pielou's Evenness

##### Which categorical sample metadata columns are most strongly associated with the differences in microbial community evenness? 
Nearly everything but Colony.... 
Surprised by one Tank result: H1 vs H2, I wouldn't have expected those to be different.

##### Are these differences statistically significant?
Pae
- control (n=8) vs peak (n=8), pvalue (0.05)
- env (n=4) vs. peak (n=8), pvalue (0.006)

Temp
- ambient (n=12) vs hot (n=8), pvalue (0.02)

PaeTemp
- control-ambient (n=4) vs peak-ambient (n=4), pvalue (0.02)
- control-ambient (n=4) vs peak-hot (n=4), pvalue (0.02)
- env-ambient (n=4) vs peak-ambient (n=4), pvalue (0.02)
- env-ambient (n=4) vs peak-hot (n=4), pvalue (0.02)
  
Colony
- no significantly different groups

Tank
- A1 (n=4) vs H1 (n=5), pvalue (0.02)
- A1 (n=4) vs KB (n=4), pvalue (0.04)
- A2 (n=4) vs H1 (n=5), pvalue (0.02)
- H1 (n=5) vs H2 (n=3), pvalue (0.05)
- H1 (n=5) vs KB (n=4), pvalue (0.01)

In [ ]:
Visualization.load('../output/evenness-group-significance.qzv')

let's use the Shannon vector in the output directory to create a visualization of alpha diversity across samples.

In [52]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity ../output/diversity-core/shannon_vector.qza \
    --m-metadata-file ../data/treatment-only-metadata.tsv \
    --o-visualization ../output/diversity-core/alpha_groups.qzv

Saved Visualization to: ../output/diversity-core/alpha_groups.qzv


In [54]:
Visualization.load("../output/diversity-core/alpha_groups.qzv")

<visualization: Visualization uuid: 423d3a09-aa09-4965-b57a-8bc5f83477f0>

## Beta Diversity

Next we’ll analyze sample composition in the context of categorical metadata using PERMANOVA (first described in Anderson (2001)) using the beta-group-significance command. The following commands will test whether distances between samples within a group, such as samples from the same body site (e.g., gut), are more similar to each other then they are to samples from the other groups (e.g., tongue, left palm, and right palm). If you call this command with the --p-pairwise parameter, as we’ll do here, it will also perform pairwise tests that will allow you to determine which specific pairs of groups (e.g., tongue and gut) differ from one another, if any. This command can be slow to run, especially when passing --p-pairwise, since it is based on permutation tests. So, unlike the previous commands, we’ll run beta-group-significance on specific columns of metadata that we’re interested in exploring, rather than all metadata columns to which it is applicable. Here we’ll apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows.

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-slim.tsv \
  --m-metadata-column Pae \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-slim.tsv \
  --m-metadata-column Temp \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --m-metadata-column Temp \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/diversity-core/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

## [Atacama Soil Microbiome: Questions to Guide Data Analysis](https://docs.qiime2.org/2023.5/tutorials/atacama-soils/#paired-end-read-analysis-commands:~:text=Questions%20to%20guide%20data%20analysis)
What sample metadata or combinations of sample metadata are most strongly associated with the differences in microbial composition of the samples? Are these associations stronger with unweighted UniFrac or with Bray-Curtis? Based on what you know about these metrics, what does that difference suggest? For exploring associations between continuous metadata and sample composition, the commands qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv will be useful. These were not covered in the Moving Pictures tutorial, but you can learn about them by running them with the `--help` parameter.

## Alpha rarefaction plotting
In this section we’ll explore alpha diversity as a function of sampling depth using the qiime diversity alpha-rarefaction visualizer. This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with --p-min-depth) and the value provided as --p-max-depth. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. The number of iterations (rarefied tables computed at each sampling depth) can be controlled with --p-iterations. Average diversity values will be plotted for each sample at each even sampling depth, and samples can be grouped based on metadata in the resulting visualization if sample metadata is provided with the --m-metadata-file parameter.

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table ../output/dada2-table.qza \
  --i-phylogeny ../output/phylogeny-tree/rooted_tree.qza \
  --p-max-depth 107656 \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/alpha-rarefaction.qzv

In [ ]:
Visualization.load('../output/alpha-rarefaction.qzv')